<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/chat_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import sqlite3
from tabulate import tabulate
import datetime

db = 'chat.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+---------+------------+------------+-------+----------+
|   id |  login  |  password  |  username  |   age |   city   |
+======+=========+============+============+=======+==========+
|    1 |         |            |    Alex    |    23 | New York |
+------+---------+------------+------------+-------+----------+
|    2 |         |            |    Ann     |    22 |  Paris   |
+------+---------+------------+------------+-------+----------+


chats
+------+--------------+--------------+-------------------------+---------------------+
|   id |   chatname   |  created_by  |       description       |     created_at      |
+======+==============+==============+=========================+=====================+
|    1 |   new_chat   |     Alex     | my new chat is the best | 2023-11-17 10:14:41 |
+------+--------------+--------------+-------------------------+---------------------+
|    2 | another_chat |     Ann      |       hobby chat        | 2023-11-17 10:14:41 |
+------+--------

In [108]:
def create_db():
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    login TEXT UNIQUE,
                    password TEXT,
                    username TEXT,
                    age INTEGER,
                    city TEXT
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS chats (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    chatname TEXT,
                    created_by TEXT,
                    description TEXT,
                    created_at DATETIME
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS chat_users (
                    username TEXT,
                    chatname TEXT
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS messages (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    chatname TEXT,
                    username TEXT,
                    message TEXT,
                    created_at DATETIME
                );
            """
        )
        connection.commit()



def create_user(username, age, city):
    with sqlite3.connect(db) as connection:
      cursor = connection.cursor()
      query = """
                  INSERT INTO users (username, age, city)
                  VALUES (?, ?, ?)
              """
      cursor.execute(query, (username, age, city))
      connection.commit()


def create_chat(chatname, description, created_by):
    with sqlite3.connect(db) as connection:
      cursor = connection.cursor()
      query = """
                  INSERT INTO chats (chatname, description, created_at, created_by)
                  VALUES (?, ?, CURRENT_TIMESTAMP, ?)
              """
      cursor.execute(query, (chatname, description, created_by))
      connection.commit()


def send_message(username, chatname, message):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    INSERT INTO messages (chatname, username, message, created_at)
                    VALUES (?, ?, ?, CURRENT_TIMESTAMP)
                """
        cursor.execute(query, (chatname, username, message))
        connection.commit()


def delete_user(username):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM users
                    WHERE username = ?
                """
        cursor.execute(query, (username,))
        connection.commit()


def delete_chat(username, chatname):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM chat_users
                    WHERE username = ? AND chatname = ?
                """
        cursor.execute(query, (username, chatname))
        connection.commit()


def delete_message(username, chatname, id):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM messages
                    WHERE chatname = ? AND username = ? AND id = ?
                """
        cursor.execute(query, (chatname, username, id))
        connection.commit()


def get_all_users_by_city(city):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    SELECT username FROM users WHERE city = ?
                """
        cursor.execute(query, (city,))
        return [row for row in cursor.fetchall()]


def get_all_chat_names(username):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    SELECT chatname FROM chats WHERE created_by = ?
                """
        cursor.execute(query, (username,))
        return [row for row in cursor.fetchall()]


def get_all_messages_of_chat(chatname):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    SELECT username, message FROM messages WHERE chatname = ?
                """
        cursor.execute(query, (chatname,))
        return [row for row in cursor.fetchall()]


create_db()

# create_user("Alex", 23, "New York")
# create_user("Ann", 22, "Paris")
# create_chat("new_chat", "my new chat is the best", "Alex")
# create_chat("another_chat", "hobby chat", "Ann")
# send_message("Alex", "new_chat", "hi")
# send_message("Alex", "another_chat", "hi")
# send_message("Ann", "new_chat", "hi")
# send_message("Ann", "another_chat", "hi")

# get_all_users_by_city("New York")
# get_all_chat_names("Alex")
get_all_messages_of_chat("new_chat")

[('Alex', 'hi'), ('Ann', 'hi')]